## LTEE-FBA-Jenga.ipynb by Rohan Maddamsetti

See documentation at: 
https://cobrapy.readthedocs.io/en/latest/  

In [1]:
import matplotlib as plt
import cobra.test
import cobra
import pandas as pd
import numpy as np
from scipy import stats
from os import path
import random

In [2]:
## use Gurobi. Necessary for constructing minimal genomes!
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
cobra_config

Attribute,Description,Value
solver,Mathematical optimization solver,gurobi
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,3
cache_directory,Path for the model cache,/Users/Rohandinho/Library/Caches/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


## Functions.

In [3]:
def make_dict_of_KO_strains(anc_model, gene_vec=[]):
    deletion_cobra_models = {}
    
    gene_str_list = [str(gene_obj) for gene_obj in anc_model.genes]
    if len(gene_vec): ## then filter for the genes (strings) in gene_vec.
        gene_str_list = [x for x in gene_str_list if x in gene_vec]
        
    for genestr in gene_str_list:
        cur_KO_model = anc_model.copy()
        cur_KO_model.id = '_'.join([anc_model.id, genestr, "knockout"])
        cur_gene = cur_KO_model.genes.get_by_id(genestr)
        cobra.manipulation.delete_model_genes(cur_KO_model, cur_gene)
        deletion_cobra_models[genestr] = cur_KO_model
    return deletion_cobra_models

In [4]:
def generate_LTEE_clone_cobra_models(KOed_genes_df, basic_model, using_locus_tag=False):
    
    LTEE_cobra_models = {}
    nonmutator_pops = ["Ara-5", "Ara-6", "Ara+1", "Ara+2", "Ara+4", "Ara+5"]
    mutator_pops = ["Ara-1", "Ara-2", "Ara-3", "Ara-4", "Ara+3", "Ara+6"]
    LTEE_pop_vec = nonmutator_pops + mutator_pops
    model_genes = [x.id for x in basic_model.genes]
    
    for LTEE_pop in LTEE_pop_vec:
        cur_KO_model = basic_model.copy()
        cur_KO_model.id = LTEE_pop + "_50K_A_clone"
        is_cur_pop = (KOed_genes_df["Population"] == LTEE_pop)
        
        if using_locus_tag:
            KOed_genes = [x for x in KOed_genes_df[is_cur_pop].locus_tag]
        else:
            KOed_genes = [x for x in KOed_genes_df[is_cur_pop].blattner]
        
        genes_to_remove = [cur_KO_model.genes.get_by_id(x) for x in KOed_genes if x in model_genes]
        if genes_to_remove:
            cobra.manipulation.delete_model_genes(cur_KO_model, genes_to_remove)
        
        LTEE_cobra_models[LTEE_pop] = cur_KO_model
    
    return LTEE_cobra_models

## Basic metabolic models to play with. Default objective is to maximize biomass.

In [5]:
BiGG_model_dir = "../data/BiGG-models"

# the E. coli K-12 iJO1366 model: this is the best curated complete E. coli model.
K12_model = cobra.io.load_json_model(path.join(BiGG_model_dir, "iJO1366.json"))
K12_model.id = "K12"

# the E. coli REL606 iECB_1328 model: most relevant to LTEE, but has stochiometric inconsistencies
## in hydrogen/proton conservation.
REL606_model = cobra.io.load_json_model(path.join(BiGG_model_dir, "iECB_1328.json"))
REL606_model.id = "REL606"

## The simplest well-curated model: E. coli core metabolism.
core_model = cobra.io.load_json_model(path.join(BiGG_model_dir, "e_coli_core.json"))
core_model.id = "Ecoli-core"

Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-16


In [6]:
## we are simulating media in which glucose import is limiting.
## Add an excess of thiamine to simulate DM25.
K12_medium = K12_model.medium
K12_medium['EX_thm_e'] = 1000.0
K12_model.medium = K12_medium
REL606_medium = REL606_model.medium
REL606_medium['EX_thm_e'] = 1000.0
REL606_model.medium = REL606_medium

print(K12_model.medium)
print(REL606_model.medium)
print(core_model.medium)

{'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_k_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_cbl1_e': 0.01, 'EX_ni2_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_zn2_e': 1000.0, 'EX_sel_e': 1000.0, 'EX_slnt_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_thm_e': 1000.0, 'EX_tungs_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_fe3_e': 1000.0}
{'EX_ca2_e': 1000.0, 'EX_cbl1_e': 0.01, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_fe3_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_ni2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_o2_e': 1000.0, 'EX_tungs_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_zn2_e': 1000.0, 'EX_sel_e': 1000.0, 'EX_slnt_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_thm_e': 1000.0, 'EX_h_e': 10

In [7]:
nonmutator_pops = ["Ara-5", "Ara-6", "Ara+1", "Ara+2", "Ara+4", "Ara+5"]
mutator_pops = ["Ara-1", "Ara-2", "Ara-3", "Ara-4", "Ara+3", "Ara+6"]
LTEE_pop_vec = nonmutator_pops + mutator_pops

## Question 2: Measure fitness of 50K A clones using the whole genome metabolic model.

In [8]:
inactive_genes_in_50K_A_clones = pd.read_csv(
    "../results/metabolic-enzymes/inactive-genes-in-LTEE-50K-A-clones.csv",)
inactive_genes_in_50K_A_clones

,Population,locus_tag,Gene,blattner,gene_length,product,start,end,strand
0,Ara-1,ECB_00100,mutT,b0099,390,nucleoside triphosphate pyrophosphohydrolase m...,113848,114237,1
1,Ara-1,ECB_00259,ykgI,b0303,237,hypothetical protein,290980,291216,-1
2,Ara-1,ECB_00320,ECB_00320,b0370,198,hypothetical protein,358825,359022,-1
3,Ara-1,ECB_00429,ybaL,b0478,1677,predicted transporter with NADP-binding Rossma...,473629,475305,-1
4,Ara-1,ECB_00460,ybbV,b0510,279,hypothetical protein,509662,509940,1
...,...,...,...,...,...,...,...,...,...
2175,Ara+5,ECB_03938,yjcF,b4066,1293,hypothetical protein,4260716,4262008,-1
2176,Ara+5,ECB_04094,yzfA,b4223,183,hypothetical protein,4431733,4431915,-1
2177,Ara+5,ECB_04101,mpl,b4233,1374,UDP-N-acetylmuramate:L-alanyl-gamma-D-glutamyl...,4439035,4440408,1
2178,Ara+5,ECB_04123,yjgN,b4257,1197,conserved inner membrane protein,4462763,4463959,1


In [11]:
## Generate models from K-12.
iJO1366_LTEE_models = generate_LTEE_clone_cobra_models(inactive_genes_in_50K_A_clones, K12_model)
## Generate models from REL606.
iECB_1328_LTEE_models = generate_LTEE_clone_cobra_models(inactive_genes_in_50K_A_clones, REL606_model, using_locus_tag=True)

Read LP format model from file /var/folders/v8/l0zk07vs7gs5zn3nr6f6qj5r0000gn/T/tmpxjj0f24i.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/v8/l0zk07vs7gs5zn3nr6f6qj5r0000gn/T/tmp613ehkal.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/v8/l0zk07vs7gs5zn3nr6f6qj5r0000gn/T/tmpf7ts8qba.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/v8/l0zk07vs7gs5zn3nr6f6qj5r0000gn/T/tmpcrlhlc7i.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/v8/l0zk07vs7gs5zn3nr6f6qj5r0000gn/T/tmp_o7ln7qb.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/v8/l0zk07vs7gs5zn3nr6f6qj5r0000gn/T/tmp6kgvfp0o.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP fo

## Cross-check the list of essential genes in the K12 and REL606 models with inactive genes in the 50K LTEE clones.

In [12]:
K12_essential_genes = [str(x) for x in cobra.flux_analysis.find_essential_genes(K12_model)]
REL606_essential_genes = [str(x) for x in cobra.flux_analysis.find_essential_genes(REL606_model)]

In [13]:
print(K12_essential_genes)
print(len(K12_essential_genes))
print()
print(REL606_essential_genes)
print(len(REL606_essential_genes))
## quickly generate an input file for STIMS to see if there is purifying selection on these genes in the LTEE.
REL606_ids = pd.read_csv("../results/REL606_IDs.csv")
REL606_metabolic_essential = REL606_ids[REL606_ids.locus_tag.isin(REL606_essential_genes)]
REL606_metabolic_essential.to_csv("../results/metabolic-enzymes/for_initial_STIMS_test.csv",sep=",")

['b1131', 'b0928', 'b4160', 'b3939', 'b2315', 'b1208', 'b1288', 'b0386', 'b2020', 'b2585', 'b4177', 'b2022', 'b3433', 'b0134', 'b0004', 'b3804', 'b0009', 'b2751', 'b1069', 'b3997', 's0001', 'b3634', 'b2752', 'b0154', 'b3729', 'b3058', 'b0785', 'b0826', 'b0159', 'b2599', 'b2600', 'b2329', 'b0133', 'b0524', 'b4039', 'b3805', 'b2472', 'b0087', 'b1062', 'b3941', 'b3196', 'b1094', 'b2763', 'b2762', 'b3177', 'b4040', 'b3607', 'b2764', 'b0784', 'b3187', 'b3389', 'b3018', 'b3958', 'b0907', 'b0775', 'b0131', 'b1281', 'b1693', 'b0142', 'b0052', 'b3608', 'b0421', 'b3633', 'b4006', 'b2564', 'b0029', 'b0003', 'b4013', 'b0522', 'b3843', 'b4245', 'b0523', 'b2311', 'b0185', 'b2320', 'b2021', 'b2316', 'b3359', 'b3642', 'b3730', 'b0915', 'b3256', 'b2574', 'b3040', 'b3255', 'b1415', 'b3368', 'b2750', 'b0750', 'b2400', 'b1662', 'b0415', 'b3176', 'b2507', 'b0639', 'b0908', 'b0166', 'b1260', 'b1261', 'b2687', 'b0774', 'b2615', 'b0475', 'b0777', 'b0109', 'b2780', 'b0720', 'b0073', 'b2530', 'b0369', 'b3807', 

In [14]:
inactive_genes_in_50K_A_clones[inactive_genes_in_50K_A_clones.blattner.isin(K12_essential_genes)]

,Population,locus_tag,Gene,blattner,gene_length,product,start,end,strand
432,Ara-3,ECB_01370,aldA,b1415,1440,aldehyde dehydrogenase A NAD-linked,1457563,1459002,1
800,Ara-4,ECB_01094,tmk,b1098,642,thymidylate kinase,1169693,1170334,1
1106,Ara-5,ECB_02596,ispF,b2746,480,2-C-methyl-D-erythritol 24-cyclodiphosphate sy...,2766162,2766641,-1
1597,Ara+2,ECB_02597,ispD,b2747,711,2-C-methyl-D-erythritol 4-phosphate cytidylylt...,2766641,2767351,-1


In [15]:
inactive_genes_in_50K_A_clones[inactive_genes_in_50K_A_clones.locus_tag.isin(REL606_essential_genes)]

,Population,locus_tag,Gene,blattner,gene_length,product,start,end,strand
432,Ara-3,ECB_01370,aldA,b1415,1440,aldehyde dehydrogenase A NAD-linked,1457563,1459002,1
800,Ara-4,ECB_01094,tmk,b1098,642,thymidylate kinase,1169693,1170334,1
1106,Ara-5,ECB_02596,ispF,b2746,480,2-C-methyl-D-erythritol 24-cyclodiphosphate sy...,2766162,2766641,-1
1558,Ara+2,ECB_02356,tktB,b2465,2004,transketolase 2 thiamin-binding,2500464,2502467,1
1597,Ara+2,ECB_02597,ispD,b2747,711,2-C-methyl-D-erythritol 4-phosphate cytidylylt...,2766641,2767351,-1


## Analyze essential genes in the LTEE 50K metabolic networks.

#### Jenga hypothesis predicts that evolved LTEE 50K metabolic networks should show variation in metabolic essentiality.
- the genes essential across all viable networks are simple the genes essential to the base REL606_model.
- make a table of the genes essential in particular genomes, but not overall. This result is small enough that it doesn't need a figure.

In [16]:
for pop, cur_model in iJO1366_LTEE_models.items():
    print(pop, cur_model.slim_optimize())

Ara-5 0.0
Ara-6 0.9824915382682922
Ara+1 0.9818549149161437
Ara+2 0.0
Ara+4 0.9824915382682928
Ara+5 0.9824765425792205
Ara-1 0.9824915382682908
Ara-2 0.9824915382682917
Ara-3 0.0
Ara-4 0.0
Ara+3 0.9824915382682917
Ara+6 0.9824915382682919


In [17]:
for pop, cur_model in iECB_1328_LTEE_models.items():
    print(pop, cur_model.slim_optimize())

Ara-5 0.0
Ara-6 0.9825981902001607
Ara+1 0.9819614286711661
Ara+2 0.0
Ara+4 0.9825981902001641
Ara+5 0.0
Ara-1 0.9825981902001673
Ara-2 0.9825981902001644
Ara-3 0.0
Ara-4 0.0
Ara+3 0.9825981902001644
Ara+6 0.9825981902001668


In [18]:
LTEEpop_to_essential_genes = {pop:[str(x) for x in cobra.flux_analysis.find_essential_genes(cur_model)] for pop,cur_model in iECB_1328_LTEE_models.items()}

In [19]:
## find the intersection of essential genes in all networks, skipping the nonviable networks for now.
pops_to_examine = [pop for pop in LTEEpop_to_essential_genes.keys() if pop not in ["Ara-5", "Ara+2", "Ara+5",
                                                                                   "Ara-3","Ara-4"]]

essential_intersection = set.intersection(*[set(LTEEpop_to_essential_genes[x]) for x in pops_to_examine])
print(len(essential_intersection))
idiosyncratic_essentials = {x:set(set(LTEEpop_to_essential_genes[x]) - essential_intersection) for x in pops_to_examine}
print(idiosyncratic_essentials)
idiosyncratic_essentials_set = set.union(*[idio_set for _, idio_set in idiosyncratic_essentials.items()])

202
{'Ara-6': {'ECB_03708', 'ECB_01730'}, 'Ara+1': {'ECB_02846', 'ECB_03708', 'ECB_02847', 'ECB_01182', 'ECB_02848', 'ECB_00425'}, 'Ara+4': {'ECB_00070', 'ECB_01102', 'ECB_00069', 'ECB_00068'}, 'Ara-1': {'ECB_01836', 'ECB_01730'}, 'Ara-2': {'ECB_00070', 'ECB_02400', 'ECB_03891', 'ECB_01102', 'ECB_00069', 'ECB_00068'}, 'Ara+3': {'ECB_00002', 'ECB_02400', 'ECB_03555', 'ECB_03554'}, 'Ara+6': set()}


In [20]:
REL606_idiosyncratic_essential = REL606_ids[REL606_ids.locus_tag.isin(idiosyncratic_essentials_set)]
REL606_idiosyncratic_essential

,Gene,locus_tag,blattner,gene_length,product,start,end,strand
1,thrA,ECB_00002,b0002,2463,bifunctional aspartokinase I/homeserine dehydr...,336,2798,1
62,yabJ,ECB_00068,b0066,699,thiamin transporter subunit,75033,75731,-1
63,thiP,ECB_00069,b0067,1611,thiamin ABC transporter membrane protein,75715,77325,-1
64,tbpA,ECB_00070,b0068,984,thiamin transporter subunit,77301,78284,-1
405,adk,ECB_00425,b0474,645,adenylate kinase,469242,469886,1
1059,ycfN,ECB_01102,b1106,825,thiamin kinase,1177829,1178653,1
1134,prsA,ECB_01182,b1207,948,ribose-phosphate pyrophosphokinase,1261078,1262025,-1
1657,gdhA,ECB_01730,b1761,1344,glutamate dehydrogenase,1819599,1820942,1
1762,ntpA,ECB_01836,b1865,453,dATP pyrophosphohydrolase,1926950,1927402,-1
2301,guaB,ECB_02400,b2508,1467,inositol-5-monophosphate dehydrogenase,2553423,2554889,-1


In [21]:
REL606_intersection_essential = REL606_ids[REL606_ids.locus_tag.isin(essential_intersection)]
pd.options.display.max_rows = None
REL606_intersection_essential

,Gene,locus_tag,blattner,gene_length,product,start,end,strand
2,thrB,ECB_00003,b0003,933,homoserine kinase,2800,3732,1
3,thrC,ECB_00004,b0004,1287,threonine synthase,3733,5019,1
8,mogA,ECB_00009,b0009,588,molybdenum cofactor biosynthesis protein,9304,9891,1
24,ribF,ECB_00029,b0025,942,putative regulator,25479,26420,1
28,ispH,ECB_00033,b0029,951,4-hydroxy-3-methylbut-2-enyl diphosphate reduc...,30349,31299,1
30,dapB,ECB_00035,b0031,822,dihydrodipicolinate reductase,32446,33267,1
50,pdxA,ECB_00056,b0052,990,4-hydroxythreonine-4-phosphate dehydrogenase,56622,57611,-1
67,leuD,ECB_00073,b0071,606,isopropylmalate isomerase small subunit,81652,82257,-1
68,leuC,ECB_00074,b0072,1401,isopropylmalate isomerase large subunit,82268,83668,-1
69,leuB,ECB_00075,b0073,1092,3-isopropylmalate dehydrogenase,83671,84762,-1


In [22]:
essential_intersection - set(REL606_essential_genes) ## This is the empty set.

set()

## TODO: Analyze minimal genomes. See Pal et al. (2006) for guidance.  
Examine variation in essential genes, based on the definition in the source code for cobra.flux_analysis.variability.

In [38]:
test = REL606_model

In [49]:
for i, rxn in enumerate(test.reactions):
    my_string = rxn.name
    my_string = my_string.strip()
    print(i, my_string)

0 D-Allose exchange
1 Allantoin exchange
2 N-Acetyl-D-glucosamine(anhydrous)N-Acetylmuramic acid exchange
3 L-Alanine exchange
4 AMP exchange
5 D-Alanyl-D-alanine exchange
6 L-Arabinose exchange
7 Aerobactin minus Fe3 exchange
8 Arbutin exchange
9 L-Arginine exchange
10 Aerobactin exchange
11 L-Asparagine exchange
12 L-Ascorbate exchange
13 Glycerophosphoserine exchange
14 Arsenite exchange
15 L-Aspartate exchange
16 Biotin exchange
17 Butyrate (n-C4:0) exchange
18 N-Acetylmuramate exchange
19 Sink needed to allow p-Cresol to leave system
20 Sink needed to allow 5'-deoxyribose to leave system
21 Sink needed to allow aminoacetaldehyde to leave system
22 Butanesulfonate exchange
23 Calcium exchange
24 Sink needed to allow S-Adenosyl-4-methylthio-2-oxobutanoate to leave system
25 Cobinamide exchange
26 Sink needed to allow (2R,4S)-2-methyl-2,3,3,4-tetrahydroxytetrahydrofuran to leave system
27 D-Glucose 6-phosphate exchange
28 Sink needed to allow oxamate to leave system
29 E. coli biomas

696 L-arginine transport via ABC system (periplasm)
697 Lipoate exchange
698 L-arginine transport out via proton antiport (cytoplasm to periplasm)
699 L-arginine transport via diffusion (extracellular to periplasm)
700 L-aspartase
701 Aspartate transaminase
702 Aspartyl-tRNA synthetase
703 L-aspartate transport via ABC system (periplasm)
704 Aspartate transport via proton symport (2 H) (periplasm)
705 Aspartate-semialdehyde dehydrogenase
706 L-ascorbate 6-phosphate lactonase
707 L-ascorbate transport via PEP:Pyr PTS (periplasm)
708 L-ascorbate transport via diffusion (extracellular to periplasm)
709 L-asparaginase
710 L-asparate transport via proton symport (3 H) (periplasm)
711 L-aspartate transport in via proton symport (periplasm)
712 L-aspartate transport in via proton symport (periplasm)
713 L-asparaginase
714 L-Lysine exchange
715 Asparagine synthase (glutamine-hydrolysing)
716 Asparagine synthetase
717 Asparaginyl-tRNA synthetase
718 L-aspartate transport via diffusion (extracel

1196 Fructose-1-phosphate kinase
1197 Fructoselysine phosphate deglycase
1198 Fructoselysine 3-epimerase
1199 Fructoselysine Kinase
1200 Glycerol-3-phosphate acyltransferase (C14:1)
1201 Glycerol-3-phosphate acyltransferase (C16:0)
1202 Glycerol-3-phosphate acyltransferase (C16:1)
1203 Glycerol-3-phosphate acyltransferase (C18:0)
1204 Glycerol-3-phosphate acyltransferase (C18:1)
1205 Sn-glycerol-3-phosphocholine transport via ABC system (periplasm)
1206 Fructoselysine transport via proton symport (periplasm)
1207 Fructoselysine transporter via diffusion (extracellular)
1208 D-fructuronate transport via proton symport, reversible (periplasm)
1209 D-fructuronate transport via diffusion (extracellular)
1210 Glycero-3-phosphocholine transport via diffusion (extracellular to periplasm)
1211 Glycerol-3-phosphate dehydrogenase (NADP)
1212 Fructose transport via PEP:Pyr PTS (f6p generating) (periplasm)
1213 Dihydrodipicolinate reductase (NADPH)
1214 D-fructose transport via PEP:Pyr PTS (peripl

1737 3-methyl-2-oxobutanoate hydroxymethyltransferase
1738 Malate oxidase
1739 Murein L,D-carboxypeptidase (murein4p4p) (periplasm)
1740 Murein L,D-carboxypeptidase (murein5p5p) (periplasm)
1741 Murein L,D-carboxypeptidase (murein4p3p) (periplasm)
1742 Murein L,D-carboxypeptidase (murein5px3p) (periplasm)
1743 Murein L,D-endopeptidase (murein3px3p) (periplasm)
1744 Murein L,D-endopeptidase (murein5px3p) (periplasm)
1745 Maltodextrin glucosidase (maltotriose)
1746 Molybdopterin adenylyltransferase
1747 Maltodextrin glucosidase (maltotetraose)
1748 Lysophospholipase L1 (2-acylglycerophosphoethanolamine, n-C14:0) (periplasm)
1749 Maltodextrin glucosidase (maltopentaose)
1750 Murein polymerizing transglycosylase
1751 Murein polymerizing transglycosylase 2 (three linked units)
1752 Molybdopterin synthase
1753 Maltodextrin glucosidase (maltohexaose)
1754 Maltodextrin glucosidase (maltoheptaose)
1755 Molybdopterin synthase sulfurylase
1756 Lysophospholipase L1 (2-acylglycerophosphoethanolamin

2367 DTDPglucose 4,6-dehydratase
2368 Tetraacyldisaccharide 4'kinase
2369 Tetrahydrodipicolinate succinylase
2370 Thiol:disulfide reductase (DsbC)
2371 Lysine decarboxylase
2372 Thiol:disulfide reductase (DsbG)
2373 Tagatose-bisphosphate aldolase
2374 Tetrahydrofolate aminomethyltransferase
2375 Hydrogen peroxide reductase (thioredoxin)
2376 Thymidine transport in via proton symport (periplasm)
2377 Thymidine transport via diffusion (extracellular to periplasm)
2378 Thiamine transport via ABC system (periplasm)
2379 NAD(P) transhydrogenase (periplasm)
2380 Lysyl-tRNA synthetase
2381 Thiamine transport via diffusion (extracellular to periplasm)
2382 Tartronate semialdehyde reductase
2383 L-allo-Threonine Aldolase
2384 L-lysine transport via ABC system (periplasm)
2385 Threonine aldolase
2386 Thiosulfate transport via ABC system (periplasm)
2387 Thiosulfate transport via diffusion (extracellular to periplasm)
2388 L-threonine dehydrogenase
2389 L-threonine deaminase
2390 Tetradecanoate t

In [33]:
for g in test.genes:
    if g.functional:
        print(g)

ECB_03432
ECB_01089
ECB_02248
ECB_01348
ECB_01091
ECB_01219
ECB_01220
ECB_01221
ECB_01218
ECB_01307
ECB_01358
ECB_02960
ECB_00004
ECB_03268
ECB_03083
ECB_03082
ECB_00741
ECB_00933
ECB_02142
ECB_01167
ECB_01217
ECB_04132
ECB_00238
ECB_03726
ECB_03871
ECB_04131
ECB_00158
ECB_03907
ECB_03906
ECB_03062
ECB_03904
ECB_03905
ECB_02558
ECB_03908
ECB_04085
ECB_01894
ECB_04126
ECB_00232
ECB_00936
ECB_02415
ECB_01090
ECB_02683
ECB_03671
ECB_02684
ECB_01610
ECB_03423
ECB_02733
ECB_01237
ECB_01238
ECB_01279
ECB_03403
ECB_02518
ECB_00743
ECB_01401
ECB_01132
ECB_02519
ECB_00053
ECB_02149
ECB_02148
ECB_00398
ECB_02151
ECB_02722
ECB_03736
ECB_01012
ECB_02266
ECB_00120
ECB_02150
ECB_02929
ECB_00178
ECB_01215
ECB_00305
ECB_00958
ECB_00972
s0001
ECB_00362
ECB_00301
ECB_01421
ECB_00064
ECB_01902
ECB_00152
ECB_00183
ECB_00151
ECB_03114
ECB_00150
ECB_02241
ECB_03606
ECB_03113
ECB_03941
ECB_02565
nagK
ECB_03662
ECB_00636
ECB_01097
ECB_02316
ECB_02317
ECB_02315
ECB_03844
ECB_02666
ECB_00080
ECB_00079
ECB_03554